In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np
from FoodItem import FoodItem


cursor = get_cursor()
# Start the timer
start = time.perf_counter()

In [2]:
user_id = (2,)
foodchecks = cursor.callproc("GetFoodChecksByID", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


In [3]:
incoming_data = {"latitude": "50.397120", "longitude": "-4.036932", "max-distance": "10", "isForChild": 0}
foodchecks = cursor.callproc("GetRestaurantsWithinDistance", (incoming_data["latitude"], incoming_data["longitude"], incoming_data["max-distance"]))
df = None

for result in cursor.stored_results():
    df = pd.DataFrame(result)

if(df.size < 1):
    raise Exception("No restaurants found within specified distance")
df.columns = ["RestaurantID", "IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy", "FoodID", "FoodName", "FoodNameShort", "IsChildMenu", "FoodTagID"]
print(df)

    RestaurantID  IsVegetarian  IsVegan  IsHalal  IsKosher  HasLactose  \
0              1             0        0        0         0           1   
1              1             0        0        0         0           1   
2              1             0        0        0         0           1   
3              1             0        0        0         0           1   
4              1             0        0        0         0           1   
..           ...           ...      ...      ...       ...         ...   
75             6             0        0        0         0           1   
76             6             0        0        0         0           1   
77             6             0        0        0         0           1   
78             6             0        0        0         0           1   
79             6             0        0        0         0           1   

    HasNuts  HasGluten  HasEgg  HasSoy  FoodID                  FoodName  \
0         1          1       1     

In [4]:
if checks[0]:
    df = df.drop(df[df["IsVegetarian"] != checks[0]].index)
if checks[1]:
    df = df.drop(df[df["IsVegan"] != checks[1]].index)
if checks[2]:
    df = df.drop(df[df["IsHalal"] != checks[2]].index)
if checks[3]:
    df = df.drop(df[df["IsKosher"] != checks[3]].index)
if checks[4]:
    df = df.drop(df[df["HasLactose"] != checks[4]].index)
if checks[5]:
    df = df.drop(df[df["HasNuts"] != checks[5]].index)
if checks[6]:
    df = df.drop(df[df["HasGluten"] != checks[6]].index)
if checks[7]:
    df = df.drop(df[df["HasEgg"] != checks[7]].index)
if checks[8]:
    df = df.drop(df[df["HasSoy"] != checks[8]].index)
if incoming_data["isForChild"]:
    df = df.drop(df[df["IsChildMenu"] != 1].index)

In [5]:
df

,RestaurantID,IsVegetarian,IsVegan,IsHalal,IsKosher,HasLactose,HasNuts,HasGluten,HasEgg,HasSoy,FoodID,FoodName,FoodNameShort,IsChildMenu,FoodTagID
0,1,0,0,0,0,1,1,1,1,1,1,Vegan Calzone,Vegan Calzone,0,1
1,1,0,0,0,0,1,1,1,1,1,1,Vegan Calzone,Vegan Calzone,0,2
2,1,0,0,0,0,1,1,1,1,1,1,Vegan Calzone,Vegan Calzone,0,4
3,1,0,0,0,0,1,1,1,1,1,44,Sweet Fire Caulliflower,Fire Caulliflower,0,21
4,1,0,0,0,0,1,1,1,1,1,45,Reverse Pizz,Reverse Pizza,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,6,0,0,0,0,1,1,1,1,1,56,Crispy Fish,Crisp Fish,0,21
76,6,0,0,0,0,1,1,1,1,1,57,Seafood Bucket,Bucket,0,9
77,6,0,0,0,0,1,1,1,1,1,58,Panang Fish Curry,Panang Curry,0,9
78,6,0,0,0,0,1,1,1,1,1,59,Stir fried Prawns,Stir Prawns,0,9


In [6]:
food_items = []

for food_id in df.FoodID.unique():
    # For each unique food id
    # Extract that part of the dataframe, and create a new food item from it
    food_items.append(FoodItem(df[df["FoodID"] == food_id]))

In [7]:
end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")

Done in 0.3281 seconds


In [8]:
for food_item in food_items:
    food_item.printInfo()

ID: 1 - (<class 'int'>)
FoodTags: [1, 2, 4] - (<class 'list'>)
Restaurant ID: 1
ID: 44 - (<class 'int'>)
FoodTags: [21] - (<class 'list'>)
Restaurant ID: 1
ID: 45 - (<class 'int'>)
FoodTags: [2, 4] - (<class 'list'>)
Restaurant ID: 1
ID: 46 - (<class 'int'>)
FoodTags: [32] - (<class 'list'>)
Restaurant ID: 1
ID: 47 - (<class 'int'>)
FoodTags: [4, 32] - (<class 'list'>)
Restaurant ID: 1
ID: 48 - (<class 'int'>)
FoodTags: [1, 6, 7, 8] - (<class 'list'>)
Restaurant ID: 1
ID: 49 - (<class 'int'>)
FoodTags: [1, 27] - (<class 'list'>)
Restaurant ID: 1
ID: 50 - (<class 'int'>)
FoodTags: [21, 32] - (<class 'list'>)
Restaurant ID: 1
ID: 2 - (<class 'int'>)
FoodTags: [5, 6, 7, 8] - (<class 'list'>)
Restaurant ID: 2
ID: 5 - (<class 'int'>)
FoodTags: [9, 11, 12] - (<class 'list'>)
Restaurant ID: 2
ID: 6 - (<class 'int'>)
FoodTags: [13, 15, 16, 17] - (<class 'list'>)
Restaurant ID: 2
ID: 7 - (<class 'int'>)
FoodTags: [18, 20, 21] - (<class 'list'>)
Restaurant ID: 2
ID: 8 - (<class 'int'>)
FoodTags: